# S1W2D2：神经网络

**今日目标**：透彻理解模型训练的完整流程，能用自己的话清晰地向面试官解释从“犯错”到“改正”的全部过程。

## 定义目标 - 损失函数（Loss Function）

如果不知道“好”与“坏”的标准，模型就无法学习。损失函数就是这个标准，它告诉我们模型当前“错得有多离谱”。

- 核心作用：将模型的预测结果与真实标签进行比较，输出一个**标量（单个数值）**来量化它们之间的差距。这个数值越大，说明模型错得越严重。
- 训练目标: 我们的整个训练过程，就是为了调整参数（$w$ 和 $b$），让这个损失值尽可能地小。

两种核心的损失函数（面试高频）

1. 均方误差损失（Mean Squared Error，MSE Loss）：
  - **适用场景**：回归任务（预测连续值，如房价、温度）。
  - **计算方式**：$L(y, \hat{y}) = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$
  - **直观理解**：计算每个样本“真实值($y$)”与“预测值 ($\hat{y}$)”之差的平方，然后求平均。平方的作用是为了让差值都是正数，并且放大较大的误差。
  - **PyTorch代码**：

In [6]:
import torch
import torch.nn as nn

loss_fn = nn.MSELoss()

real_prices = torch.tensor([100.0, 150.0])
predicted_prices = torch.tensor([110.0, 145.0]) # 预测有一定误差

loss = loss_fn(predicted_prices, real_prices)
print(f"MSE Loss: {loss}") # 输出: tensor(62.5000), ((110-100)^2 + (145-150)^2) / 2 = (100+25)/2 = 62.5

MSE Loss: 62.5


2. 交叉熵损失 (Cross-Entropy Loss):

- **适用场景**: 分类任务 (预测离散类别，如 MNIST 数字识别、猫狗分类)。
- **直观理解**: 它衡量的是模型预测的“概率分布”与“真实标签的概率分布”之间的距离。如果模型对正确类别的预测概率很高（例如，图片是猫，模型预测猫的概率是 0.99），那么交叉熵损失就很小；反之则很大。
- **重要提示**: 在 PyTorch 中，nn.CrossEntropyLoss 已经内置了 Softmax 操作。因此，模型输出层不需要手动添加 Softmax 层。
- **PyTorch代码**:

In [7]:
loss_fn = nn.CrossEntropyLoss()

# 假设是3分类任务, batch_size=1
# 模型的原始输出 (logits)，不需要手动做 Softmax
model_outputs = torch.tensor([[2.0, -1.0, 0.5]]) # batch_size=1, num_classes=3
# 真实标签 (正确类别是 0)
real_labels = torch.tensor([0]) 

loss = loss_fn(model_outputs, real_labels)
print(f"Cross-Entropy Loss: {loss}")

Cross-Entropy Loss: 0.24131132662296295


## 找到方向 - 梯度下降 (Gradient Descent)

我们有了损失这个“目标”，现在需要一种方法来让损失变小。梯度下降就是这个方法。

- 核心思想 (下山比喻)：
  - **当前参数**($w, b$) 决定了你在损失函数这座“山”上的位置。
  - **损失值**就是你所在位置的海拔。
  - **梯度** (Gradient) 是一个向量，指向函数值上升最快的方向。那么，梯度的反方向就是函数值下降最快的方向，也就是“最陡的下坡方向”。
  - **学习率** (Learning Rate, $\alpha$) 就是你朝着下坡方向迈出的那一步的步长。
  - **参数更新**: 不断重复 新参数 = 旧参数 - 学习率 * 梯度 这个过程，就能一步步走到山谷（损失最小值）。
- **学习率** ($\alpha$) 的重要性 (面试高频):
  - 学习率太大: 步子迈得太大，可能会直接“跨过”山谷最低点，导致损失值在最低点附近来回震荡，甚至越来越大，无法收敛。
  - 学习率太小: 步子太小，下山速度会非常慢，导致训练时间过长。
- 核心变种：随机梯度下降 (Stochastic Gradient Descent, SGD)
  - 传统的梯度下降需要计算整个训练集的损失才能更新一次参数，当数据集很大时，这非常耗时。
  - SGD的思想是，每次只随机取一小批 (mini-batch) 数据（例如 64 张图片）来计算损失和梯度，并立即更新参数。这样做虽然每次更新的方向不一定绝对正确，但速度极快，总体上是朝着正确的方向前进的。我们现在使用的几乎都是 SGD 及其变体（如 Adam）。

## 高效计算 - 反向传播 (Backpropagation)
我们知道了要沿着“梯度”下降，但一个百万级参数的网络，如何得到损失函数对每一个参数的梯度呢？手动求导是不可能的。
- **核心作用**: 它不是一种优化算法，而是一种高效计算梯度的算法。
- **工作原理**: 基于微积分的链式法则。它从最终的损失值开始，将“误差”从输出层反向传播到隐藏层，再到输入层，沿途计算出损失对网络中每一个参数（$w$ 和 $b$）的偏导数（即梯度）。
- 你在 PyTorch 中的体验: 这个复杂的过程被封装成了一个极其简单的命令：

```python
# loss 是你计算出的损失值张量
loss.backward() 

# 执行完上面这行后，所有 requires_grad=True 的参数
# (比如 a_layer.weight) 的 .grad 属性就会被填上计算好的梯度值
# print(a_layer.weight.grad)
```

- **面试要点**: 你不需要会手写反向传播的公式。但你必须清楚地知道：反向传播是为了高效计算梯度，而梯度下降是利用这些梯度来更新模型的参数。

## 补充知识：数据划分 (The Playground Rules)

为了科学地训练和评估模型，我们必须将数据集划分为三个独立的集合：

1. 训练集 (Training Set):
  - 作用: 模型学习的唯一数据来源。用于计算损失、计算梯度、更新模型参数。
  - 比喻: 学生用来学习的课本和习题集。

2. 验证集 (Validation Set):
  - 作用: 在训练过程中，用来评估模型在“未见过”的数据上的表现，以便我们调整超参数（如学习率、网络层数等），并判断模型是否过拟合。
  - 重要: 它不参与梯度的计算和参数的更新。
  - 比喻: 学生用来检验学习效果的模拟考试。考得不好，可以回去调整学习方法（调整超参数），再重新看书做题。
3. 测试集 (Test Set):
  - 作用: 在模型训练完全结束后，用来评估模型的最终性能的“终极考卷”。
  - 重要: 在整个训练和调参过程中，绝对不能使用测试集。 否则，就像是提前知道了考题答案，评估结果是虚高的、无效的。
  - 比喻: 学生的最终高考。一考定音，报告最终成绩。